Sua tarefa será  gerar a matriz termo documento, dos documentos recuperados da internet e 
imprimir esta matriz na tela. Para tanto: 
a) Considere que todas as listas de sentenças devem ser transformadas em listas de vetores, 
onde cada item será uma das palavras da sentença. 
b) Todos  os  vetores  devem  ser  unidos  em  um  corpus  único  formando  uma  lista  de  vetores, 
onde cada item será um lexema.  
c) Este único corpus será usado para gerar o vocabulário. 
d) O  resultado  esperado  será  uma  matriz  termo  documento  criada  a  partir  da  aplicação  da 
técnica bag of Words em todo o corpus.  
 

In [16]:
#Aluno: Fernando Purkott
from bs4 import BeautifulSoup
import requests
import spacy
from spacy.language import Language
import re
import string
import nltk
from nltk.corpus import stopwords



nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



@Language.component("Space")
def set_custom_boundaries(doc):
     for token in doc[:-1]:
         if token.text.__contains__('  '):
             doc[token.i+1].is_sent_start = True
     return doc
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('Space', before="parser")

def excluiNovaLinha(value):
    return ''.join(value.splitlines())



blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'style'
]

def scrape(URL):
  blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'style'
  ]

  req = requests.get(URL)
  soup = BeautifulSoup(req.content)


  txt = soup.find_all(text=True)
  clean_txt = ""

  for t in txt:
    if t.parent.name not in blacklist:
        clean_txt += '{} '.format(t)

  txt_doc = nlp(excluiNovaLinha(clean_txt))


  sentences = list(txt_doc.sents)
  return sentences

def criaVocab(sentences):
  vocab = []
  sentenceWords = []
  for s in sentences:
    for i in s:
      sentenceWords = str(i).split()
      for j in sentenceWords:
        j = j.lower()
        if j not in stop_words and j != '':
          vocab.append(j.translate(str.maketrans('', '', string.punctuation)))  
      sentenceWords = []
    vocab = unico(vocab)
    return vocab
  

def unico(words):
  seen = set()
  return [x for x in words if not (x in seen or seen.add(x))]

def vectorize(tokens, vocab):
    vector=[]
    for w in vocab:
        vector.append(tokens.count(w))
    return vector


# "https://www.ibm.com/cloud/learn/natural-language-processing"

sentences1 = scrape("https://www.ibm.com/cloud/learn/natural-language-processing")
sentences2 = scrape("https://hbr.org/2022/04/the-power-of-natural-language-processing")
sentences3 = scrape("https://link.springer.com/article/10.1007/s11042-022-13428-4")
sentences4 = scrape("https://www.techtarget.com/searchenterpriseai/definition/natural-language-processing-NLP")
sentences5 = scrape("https://monkeylearn.com/natural-language-processing/")

sentences = [sentences1, sentences2, sentences3, sentences4, sentences5]

vocab = criaVocab(sentences)

print(vocab)
for i in sentences:
  for j in i:
    tokens = str(j).split()
    vector = vectorize(tokens, vocab)
    print(vector)

    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
